# Setup

## Libraries

In [ ]:
# Data manipulation
import numpy as np
import pandas as pd

# Exploratory data analysis
import sidetable
from pandas_profiling import ProfileReport

from itertools import combinations, permutations


# Statistical packages ### & ML packages
import phik
import prince
from scipy import stats

# Data visualization
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import patches, colors
from adjustText import adjust_text

# Widget interaction
from ipywidgets import interact

## Library settings

In [ ]:
# see rc Params on https://matplotlib.org/stable/tutorials/introductory/customizing.html
# or just take a look at plt.rcParams
sns.set_theme(
    context='talk',
    style='ticks',
    font_scale=.6,
    rc={
        'figure.figsize': (12,8),
        'axes.grid': True,
        'grid.alpha': .2,
        'axes.labelpad': 20,
        'axes.titlepad': 30
    }
)

## Custom functions

In [ ]:
## -- Observed-expected plots -- ##

def bootstrap_chi2(df, cat_col1, cat_col2, expected=False, n_boostrap=1_000, ci=.95):
    ci_lower = np.round((1-ci)/2, 3)
    ci_upper = 1 - ci_lower
    
    bootstrap_values = []
    for i in range(n_boostrap):
        df_sample = df.sample(frac=1, replace=True)
        df_sample_crosstab = pd.crosstab(df_sample[cat_col1], df_sample[cat_col2])
        if expected:
            bootstrap_values.append(stats.chi2_contingency(df_sample_crosstab)[-1])
        else:
            bootstrap_values.append(df_sample_crosstab)
    
    bootstrap_values_pct = [boostrap_sample / boostrap_sample.sum() for boostrap_sample in bootstrap_values]
    
    ci_array = np.quantile(a=np.array(bootstrap_values), q=[ci_lower,ci_upper], axis=0).round().astype(int)
    ci_array_pct = np.quantile(a=np.array(bootstrap_values_pct), q=[ci_lower,ci_upper], axis=0).round(2)
    
    return ci_array, ci_array_pct


def make_heatmap_freqlabel(df, index, columns, expected=False, bootstrap=False, **kwargs):
    df_crosstab = pd.crosstab(df[index], df[columns])
    
    if expected:
        df_crosstab = (
            pd.DataFrame(stats.chi2_contingency(df_crosstab)[-1], index=df_crosstab.index, columns=df_crosstab.columns)
            .round().astype(int)
        )
    
    df_crosstab_str = df_crosstab.astype(str)
    df_crosstab_str_pct = (df_crosstab / df_crosstab.sum().sum()*100).round().astype(int).astype(str)+'%'
    
    if bootstrap:
        ci = bootstrap_chi2(df, index, columns, expected, **kwargs)
        df_crosstab_str = df_crosstab_str + '\n(' + ci[0][0].astype(str) + ' - ' + ci[0][1].astype(str) + ')'
        
        ci_lower_pct = (ci[1][0]*100).astype(int).astype(str)
        ci_upper_pct = (ci[1][1]*100).astype(int).astype(str)
        df_crosstab_str_pct = df_crosstab_str_pct + '\n(' + ci_lower_pct + '% - ' + ci_upper_pct + '%)'
        
    return df_crosstab_str + '\n\n' + df_crosstab_str_pct


## -- Cramer's V -- ##

def calculate_cramerV(df, index, columns):
    df_crosstab = pd.crosstab(df[index], df[columns]).values
    X2 = stats.chi2_contingency(df_crosstab, correction=False)[0]
    minimum_dimension = min(df_crosstab.shape)-1
    N = np.sum(df_crosstab)
    result = np.sqrt(X2 / (N*minimum_dimension))
    return result


## -- Entropy Functions-- ##

def calculate_entropy(array):
    norm_counts = pd.Series(array).value_counts(normalize=True)
    norm_counts *= np.log2(norm_counts)
    return -np.sum(norm_counts)


def information_gain(df, target_col, split_col):
    parent_entropy = calculate_entropy(df[target_col])
    child_proportions = df[split_col].value_counts(normalize=True)
    child_weighted_entropies = np.sum([pct*df.query(f"{split_col}==@category")[target_col].pipe(calculate_entropy) for category,pct in child_proportions.iteritems()])
    information_gain = parent_entropy - child_weighted_entropies
    return parent_entropy, child_weighted_entropies, information_gain


def calculate_child_entropies(df, target_col, split_col):
    child_proportions = df[split_col].value_counts(normalize=True)
    child_entropies = {
        category: (
            proportion,
            df.query(f"{split_col}==@category")[target_col].pipe(calculate_entropy)
        )
        for category,proportion in child_proportions.iteritems()
    }
    
    df_child_entropies = (
        pd.DataFrame(child_entropies, index=['proportion','entropy'])
        .transpose()
        .sort_values('proportion')
        .assign(cumulative_proportion = lambda x: x.proportion.cumsum())
    )

    return df_child_entropies


def plot_child_entropy(e, ax=None):
    if ax is None:
        fig,ax = plt.subplots(figsize=(11,8))

    for i in range(e.shape[0]):
        x0 = 0 if i==0 else e.iloc[i-1, e.columns.get_loc('cumulative_proportion')]
        x1 = e.iloc[i, e.columns.get_loc('cumulative_proportion')]
        y0 = 0
        y1 = e.iloc[i, e.columns.get_loc('entropy')]
        center_x = (x0+x1) / 2
        center_y = (y0+y1) / 2
        first_letter_annotation = str(e.index[i])[0]

        rect = patches.Rectangle(xy=(x0,y0), width=x1-x0, height=y1, hatch='/', edgecolor='gray', facecolor=colors.to_rgba('gray', 0.1), lw=2)
        ax.add_patch(rect)

        ax.annotate(first_letter_annotation, (center_x, center_y), weight='bold', ha='center', va='center')
        ax.plot(center_x, center_y, 'o', ms=30, mfc='white', color='black', linewidth=4)
        ax.set(xlim=(0,1),ylim=(0,1), xlabel='Proporção', ylabel='Entropia')
        ax.set_yticks(np.arange(2,12, 2)/10)
        ax.tick_params(left=False, bottom=False)
        ax.grid(True, ls='--', alpha=.3, color='black')
        
    return ax


def plot_parent_entropy(e, ax=None):
    if ax is None:
        fig,ax = plt.subplots(figsize=(11,8))

    rect = patches.Rectangle(xy=(0,0), width=1, height=e, hatch='/', edgecolor='gray', facecolor=colors.to_rgba('gray', 0.1), lw=2)
    ax.add_patch(rect)

    ax.set(xlim=(0,1),ylim=(0,1), xlabel='Proporção', ylabel='Entropia')
    ax.set_yticks(np.arange(2,12, 2)/10)
    ax.tick_params(left=False, bottom=False)
    ax.grid(True, ls='--', alpha=.3, color='black')

    return ax

## Load dataset

The Telco customer churn data contains information about a fictional telco company that provided home phone and Internet services to 7043 customers in California in Q3.  
It indicates which customers have left, stayed, or signed up for their service.

Downladed from: https://www.kaggle.com/datasets/blastchar/telco-customer-churn

In [ ]:
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv', index_col='customerID')

In [ ]:
print('Dataframe has {} rows and {} columns.\n'.format(*df.shape))
display(df.head())

In [ ]:
df.info()

In [ ]:
numerical_columns = ['tenure','MonthlyCharges','TotalCharges']
categorical_columns = df.columns[~df.columns.isin(numerical_columns)].tolist()

df[categorical_columns] = df[categorical_columns].astype('category')

In [ ]:
df.info()

# MAIN

## EDA

In [ ]:
df.stb.freq(['Churn'])

In [ ]:
df.profile_report(interactions=None)

In [ ]:
target_col = categorical_columns.pop(categorical_columns.index('Churn'))

## Chi2

### Chi-squared test

$$\chi^2 = \sum_i{\frac{(O_i-E_i)^2}{E_i}}$$

#### Observed frequencies

In [ ]:
index = 'PaperlessBilling'
columns = 'Churn'

df_crosstab = pd.crosstab(df[index], df[columns])
df_crosstab

#### Expected frequencies

In [ ]:
stats.chi2_contingency(df_crosstab)

In [ ]:
pd.DataFrame(
    data=stats.chi2_contingency(df_crosstab)[-1],
    index=df_crosstab.index,
    columns=df_crosstab.columns
)

Exemplo de quando os valores observados são iguais aos esperados:

In [ ]:
df_crosstab = pd.DataFrame(
    np.array([100]*4).reshape((2,2))
)

df_crosstab

In [ ]:
stats.chi2_contingency(df_crosstab)

#### Teste qui-quadrado

Aplicar teste para todas variáveis categóricas do dataset:

In [ ]:
df_styled = (
    pd.DataFrame(
        {cat_col: stats.chi2_contingency(pd.crosstab(df[cat_col], df.Churn), correction=True)[:2] for cat_col in categorical_columns},
        index=['chi2','pvalue']
    )
    .transpose()
    .sort_values('chi2', ascending=False)
    .style.background_gradient(cmap='Oranges')
    
)

display(df_styled)

#### Observed vs Expected heatmap

In [ ]:
@interact(index=categorical_columns, columns=['Churn'])
def plot_heatmap(index, columns, bootstrap=False):
    df_absfreq = pd.crosstab(df[index], df[columns])
    df_absfreq_expected = pd.DataFrame(stats.chi2_contingency(df_absfreq)[-1], index=df_absfreq.index, columns=df_absfreq.columns).round().astype(int)

    plot_title = f'{index} vs {columns}'
    fig, ax = plt.subplots(ncols=2, figsize=(12,6))

    ax1 = sns.heatmap(df_absfreq, cmap='Blues', annot=make_heatmap_freqlabel(df, index, columns), fmt='', cbar=False, ax=ax[0],linewidths=.5)
    ax1.text(x=0.5, y=1.1, s=plot_title, fontsize='large', weight='bold', ha='center', va='bottom', transform=ax1.transAxes)
    ax1.text(x=0.5, y=1.04, s='Observed values', fontsize='small', alpha=0.75, ha='center', va='bottom', transform=ax1.transAxes)
    ax1.tick_params(left=False)

    ax2 = sns.heatmap(df_absfreq_expected, cmap='Blues', annot=make_heatmap_freqlabel(df, index, columns, bootstrap=bootstrap, expected=True), cbar=False, fmt='', ax=ax[1], linewidths=.5)
    ax2.text(x=0.5, y=1.1, s=plot_title, fontsize='large', weight='bold', ha='center', va='bottom', transform=ax2.transAxes)
    ax2.text(x=0.5, y=1.04, s='Expected values under independent-association hypothesis', fontsize='small', alpha=0.75, ha='center', va='bottom', transform=ax2.transAxes)
    ax2.tick_params(left=False)
    ax2.set_ylabel(None)
    ax2.set_yticks([])

    plt.tight_layout(w_pad=3)
    plt.show()

#### Chi-squared distribution

In [ ]:
@interact(index=categorical_columns, columns=['Churn'])
def plot_chi2(index, columns):
    # Get observed chi2 and dof
    chi2, p, dof, expected = stats.chi2_contingency(pd.crosstab(df[index], df[columns]))
    # Get theoretical chi2 distribution
    theorical_chi2_dist = np.random.chisquare(dof, df.shape[0])

    # Plot
    plt.figure(figsize=(9,6))
    ax = sns.histplot(theorical_chi2_dist, kde=True, color='orange', alpha=.3, stat='density')
    ax.axvline(chi2, ls='--', color='black', ymax=.9, label=f'$\chi^2$ = {chi2:.2f}')
    plt.legend(loc='upper right', prop={'size':'small'}, frameon=False)
    
    ax.set_title(f'Theoretical $\chi^2$ distribution for {index} vs {columns}', y=1.05, weight='bold', size=14)
    ax.annotate(f'P(X > {chi2:.2f}) = {1-stats.chi2.cdf(chi2, dof):.4f}', xy=(.5,1.06), xycoords='axes fraction', ha='center')
    
    sns.despine(offset=10, trim=True, ax=ax)
    plt.show()

## Cramer's V

Fórmula:

$$
\text{Cramer's V} = \sqrt{\frac{\chi^2}{N(k-1)}}
$$

onde:  
$N$: soma dos valores da tabela de contingência  
$k$ = valor mínimo de dimensão (quantidade de linhas vs quantidade de colunas)

In [ ]:
df_cramer = pd.DataFrame([(var1, var2, calculate_cramerV(df, var1, var2)) for var1,var2 in permutations(categorical_columns+['Churn'], r=2)], columns=['var1','var2','cramerV'])
df_cramer = df_cramer.pivot(index='var1', columns='var2', values='cramerV')
df_cramer

In [ ]:
fig,ax = plt.subplots(figsize=(12,8))

mask = np.zeros(df_cramer.shape).astype(bool)
mask[np.triu_indices_from(mask)] = True

sns.heatmap(df_cramer, cmap='Oranges', mask=mask, square=False, vmin=0, vmax=1, linewidths=1,
            annot=True, fmt='.2f', annot_kws={'fontsize':'small'},
            cbar=True, cbar_kws={'orientation':'vertical', 'shrink': .7}, ax=ax)
ax.set(xlabel=None, ylabel=None)
plt.yticks(rotation=0)
plt.show()

In [ ]:
pd.crosstab(df['PhoneService'], df['MultipleLines'])

## ANACOR

### Correspondence Analysis (CA)

In [ ]:
@interact(cat_col1=categorical_columns+[target_col], cat_col2=categorical_columns+[target_col])
def plot_ca(cat_col1, cat_col2, ajust_labels=False):
    df_crosstab = pd.crosstab(df[cat_col1], df[cat_col2])
    ca = prince.ca.CA().fit(df_crosstab)

    ax = ca.plot_coordinates(df_crosstab, figsize=(10,6))
    # sns.despine(offset=10, trim=False)

    if ajust_labels:
        adjust_text(ax.texts)

    plt.show()

### Multiple Correspondence Analysis (MCA)

In [ ]:
X = df[categorical_columns+[target_col]]

In [ ]:
mca = prince.MCA().fit(X)

In [ ]:
sns.set_palette('tab20')
ax = mca.plot_coordinates(X, figsize=(12,8), column_points_size=50, show_column_labels=True)
# adjust_text(ax.texts)
plt.show()

## Entropy

### Entropy calculation

$$
Entropy = - \sum_{i=1}^{k}{\ p_i\ log_2(p_i)}
$$

onde $p$ é a proporção de observações para a categoria $i$

Fórmula do ganho de informação (*information gain - IG*)

$$
IG = E(P) - \sum_{i=1}{p(C)_i E(C)_i}
$$

### Entropy visualization

In [ ]:
@interact(target_col=['Churn'], split_col=categorical_columns)
def plot_entropies(target_col, split_col):
    
    e_children = calculate_child_entropies(df, target_col, split_col)
    e_parent, e_children_weighted, ig = information_gain(df, target_col, split_col)

    fig, axes = plt.subplots(ncols=2, figsize=(14,6), gridspec_kw={'width_ratios':[.5,.6]})

    plot_parent_entropy(e_parent, ax=axes[0])
    plot_child_entropy(e_children, ax=axes[1])

    axes[0].set_title(f'Entropia padrão para {target_col}', size='large', y=1.05)
    axes[0].annotate(f'Parent entropy: {e_parent:.2f}', xy=(.5,1.06), xycoords='axes fraction', ha='center', c='gray')

    axes[1].set_title(f'Entropia e prevalência de valores para {split_col}', size='large', y=1.05)
    axes[1].annotate(f'Weighted children entropy = {e_children_weighted:.2f} | Information gain = {ig:.2f}', xy=(.5,1.06), xycoords='axes fraction', ha='center', c='gray')

    plt.tight_layout(w_pad=3)
    plt.show()

### Tree-based model to assess correspondence among categorical variables

In [ ]:
import graphviz
from sklearn import tree
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text

In [ ]:
X,y = pd.get_dummies(df[categorical_columns]), df[target_col]

In [ ]:
dt = DecisionTreeClassifier(criterion='entropy', max_depth=3).fit(X,y)

In [ ]:
# Get dot data
dot_data = tree.export_graphviz(
    decision_tree=dt,
    out_file=None, 
    feature_names=dt.feature_names_in_,  
    class_names=['no','yes'],
    filled=True
)

# Draw graph
graph = graphviz.Source(dot_data, format="png") 
graph